<a href="https://colab.research.google.com/github/Alex112525/LangChain-with-LLMs/blob/main/LangChain_FewShotPromptTemplate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers einops accelerate langchain bitsandbytes xformers

In [2]:
!nvidia-smi

Wed Jul  5 03:59:55 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P8    13W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, pipeline
from langchain import PromptTemplate, LLMChain

import torch
import langchain
from IPython.display import display, Markdown

In [4]:
def printmd(string):
    display(Markdown(string))

**bertin-project/bertin-gpt-j-6B-alpaca** is an artificial intelligence language model based on the GPT-J 6B model. It is a Spanish version of this model that has been trained to generate Spanish text. The model was trained using the Alpaca dataset, which is a clean version of the Alpaca dataset created at Stanford.

In [ ]:
model = "bertin-project/bertin-gpt-j-6B-alpaca"  # The name of the model to use.

tokenizer = AutoTokenizer.from_pretrained(model)  # The tokenizer to use to encode and decode text.

pipeline = pipeline("text-generation",  # The name of the pipeline to use.
                    model=model,  # The model to use for text generation.
                    tokenizer=tokenizer,  # The tokenizer to use to encode and decode text.
                    torch_dtype=torch.bfloat16,  # The data type to use for the model's computations.
                    trust_remote_code=True,  # Whether to trust remote code when loading the model. This should only be set to True if you trust the source of the model.
                    device_map="auto",  # The device to use for the model's computations.
                    max_length=512,  # The maximum length of the text to generate.
                    do_sample=True,  # Whether to use sampling when generating text.
                    top_k=2,  # The number of candidates to keep when sampling.
                    num_return_sequences=1,  # The number of text sequences to generate.
                    eos_token_id=tokenizer.eos_token_id  # The ID of the end-of-sequence token.
                    )


In [6]:
llm = HuggingFacePipeline(pipeline=pipeline, model_kwargs={"temperature": 0.2})

## FewShotPromptTemplate

### Test 1

*Few-shot prompting* or *Context learning* is a machine learning technique that allows a model to process examples before attempting a task. This technique is used in natural language processing and is considered an emerging property of large language models.

In [18]:
from langchain import FewShotPromptTemplate

# First we make a list of examples that we want the model to learn from.
examples = [
    {"pregunta": 'Invierno es a ______ como verano es a calor', "respuesta" : "frío"},
    {"pregunta": 'Hielo es a sólido como agua es a ______', "respuesta": "líquido"},
    {"pregunta": 'Luz es a día como estrellas son a ________', "respuesta": "noche"}
]

""" Now we assemble a template for the model, as in the examples we include
response in the template there must be a space for this variable.
In template we tell it how we want to format the promt and where
to obtain the variables inside the variable examples."""

few_shot_promp = PromptTemplate(input_variables=["pregunta", "respuesta"],
                                     template = "Pregunta: {pregunta}\nRespuesta: {respuesta}")

In [19]:
examples[0]

{'pregunta': 'Invierno es a ______ como verano es a calor',
 'respuesta': 'frío'}

In [39]:
promt_examples = FewShotPromptTemplate(example_prompt=few_shot_promp,
                                       examples=examples,
                                       prefix = "Eres un asistente virtual muy bueno para los acertijos logicos.",
                                       suffix = "Rellena el espacio en blanco: {pregunta}\nRespuesta:",
                                        input_variables=["pregunta"])

In [32]:
question_1 = "Pluma es a escribir como pincel es a ________."
question_2 = "Libro es a leer como película es a ________."
question_3 = "Arcoíris es a colores como música es a ________."

In [40]:
prompt_value = promt_examples.format(pregunta=question_1)
prompt_value_2 = promt_examples.format(pregunta=question_2)
prompt_value_3 = promt_examples.format(pregunta=question_3)
print(prompt_value_3)

Eres un asistente virtual muy bueno para los acertijos logicos.

Pregunta: Invierno es a ______ como verano es a calor
Respuesta: frío

Pregunta: Hielo es a sólido como agua es a ______
Respuesta: líquido

Pregunta: Luz es a día como estrellas son a ________
Respuesta: noche

Rellena el espacio en blanco: Arcoíris es a colores como música es a ________.
Respuesta:


#### Test without FewShotPrompting

In [23]:
printmd(llm(question_1)) #"Pluma es a escribir como pincel es a ________."

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




### Respuesta:
Pintar

In [27]:
printmd(llm(question_2)) #"Libro es a leer como película es a ________."

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




### Respuesta:
Película.

In [36]:
printmd(llm(question_3)) #"Arcoíris es a colores como música es a ________."

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




### Respuesta:
Arcoíris es a tonos.

#### Test with FewShotPrompting

In [24]:
printmd(llm(prompt_value)) #"Pluma es a escribir como pincel es a ________."

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 pintar

In [30]:
printmd(llm(prompt_value_2)) #"Libro es a leer como película es a ________."

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 ver

In [35]:
printmd(llm(prompt_value_3)) #"Arcoíris es a colores como música es a ________."

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 sonido

### Test 2

In [70]:
from langchain import FewShotPromptTemplate

# First we make a list of examples that we want the model to learn from.
examples = [
    {"enunciado": "El perro ladraba muy fuerte durante toda la noche.", "resumen": "El perro ladraba."},
    {"enunciado": "El científico realizó una serie de experimentos exhaustivos para probar su teoría.", "resumen": "El científico experimentó."},
    {"enunciado": "Ana caminó bajo la lluvia, se mojó completamente y llegó empapada a su casa.", "resumen": "Ana se mojó."}
]

""" Now we assemble a template for the model, as in the examples we include
response in the template there must be a space for this variable.
In template we tell it how we want to format the promt and where
to obtain the variables inside the variable examples."""

few_shot_promp = PromptTemplate(input_variables=["enunciado", "resumen"],
                                     template = "Enunciado: {enunciado}\nResumen: {resumen}")

In [82]:
promt_examples = FewShotPromptTemplate(example_prompt=few_shot_promp,
                                       examples=examples,
                                       prefix = "Responde con el resumen de las acciones que realizo el sujeto en el enunciado.",
                                       suffix = "Enunciado: {enunciado}\nResumen:",
                                        input_variables=["enunciado"])

In [68]:
sentence = "El artista pintó un retrato detallado del paisaje montañoso utilizando una paleta de colores vibrantes."
sentence_2 = "El niño se subió al árbol, saltó de una rama a otra y finalmente alcanzó el nido de pájaros."

In [83]:
prompt_value = promt_examples.format(enunciado=sentence)
prompt_value_2 = promt_examples.format(enunciado=sentence_2)
printmd(prompt_value)

Responde con el resumen de las acciones que realizo el sujeto en el enunciado.

Enunciado: El perro ladraba muy fuerte durante toda la noche.
Resumen: El perro ladraba.

Enunciado: El científico realizó una serie de experimentos exhaustivos para probar su teoría.
Resumen: El científico experimentó.

Enunciado: Ana caminó bajo la lluvia, se mojó completamente y llegó empapada a su casa.
Resumen: Ana se mojó.

Enunciado: El artista pintó un retrato detallado del paisaje montañoso utilizando una paleta de colores vibrantes.
Resumen:

In [84]:
printmd(llm(prompt_value))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 El artista pintó un paisaje montañoso con una paleta de colores vibrante.

In [85]:
printmd(llm(prompt_value_2))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 El niño escaló el árbol.